# Moving both cleaned weekly data and forecasted weekly data into PostgreSQL

### Imports

In [2]:
import pandas as pd
from sqlalchemy import create_engine

### Load CSVs

In [6]:
# Load cleaned historical weekly data
historical = pd.read_csv("../input/named_data_cleaned.csv", parse_dates=["datum"])

# Load forecasted weekly data
forecast = pd.read_csv("../output/forecast_next_3_months.csv", parse_dates=["datum"])

# Check first few rows
print("Historical data:")
print(historical.head())
print("\nForecast data:")
print(forecast.head())

Historical data:
            datum  Antiinflammatory  Antirheumatic  Analgesics  Antipyretics  \
2014-01-05   14.0             11.67           21.3      185.95          41.0   
2014-01-12  29.33             12.68           37.9      190.70          88.0   
2014-01-19  30.67             26.34           45.9      218.40          80.0   
2014-01-26   34.0             32.37           31.5      179.60          80.0   
2014-02-02  31.02             23.35           20.7      159.88          84.0   

            Psycholeptics  Sedatives  Bronchodilators  Antihistamines  
2014-01-05            0.0       32.0              7.0             NaN  
2014-01-12            5.0       21.0              7.2             NaN  
2014-01-19            8.0       29.0             12.0             NaN  
2014-01-26            8.0       23.0             10.0           34.00  
2014-02-02           12.0       29.0             12.0           31.02  

Forecast data:
       datum  Antiinflammatory  Antirheumatic  Analges

/var/folders/vp/2_mwt2_577d0yzkwp961ny140000gn/T/ipykernel_12778/2841515500.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  historical = pd.read_csv("../input/named_data_cleaned.csv", parse_dates=["datum"])
/var/folders/vp/2_mwt2_577d0yzkwp961ny140000gn/T/ipykernel_12778/2841515500.py:5: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  forecast = pd.read_csv("../output/forecast_next_3_months.csv", parse_dates=["datum"])


### Connect to PostgreSQL

In [10]:
# PostgreSQL connection details
user = "your_username"      # Replace with your PostgreSQL username
password = "your_password"  # Replace with your PostgreSQL password
host = "localhost"          # Or your host
port = "5432"               # Default PostgreSQL port
database = "your_database"  # Replace with your database name

# Create SQLAlchemy engine
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")

print("✅ PostgreSQL connection ready")

✅ PostgreSQL connection ready


### Push Historical Data

In [9]:
# Insert historical weekly data into PostgreSQL
historical.to_sql(
    "actual_data",      # Table name
    engine,
    if_exists="replace", # Use "append" if you want to add instead of replace
    index=False
)

print("✅ Historical weekly data inserted into 'actual_data' table")

✅ Historical weekly data inserted into 'actual_data' table


### Push Forecasted Data

In [8]:
# Insert forecasted weekly data into PostgreSQL
forecast.to_sql(
    "forecast_data",     # Table name
    engine,
    if_exists="replace", # Use "append" if needed
    index=False
)

print("✅ Forecasted weekly data inserted into 'forecast_data' table")

✅ Forecasted weekly data inserted into 'forecast_data' table


### Quick Verification

In [7]:
# Verify historical data
with engine.connect() as conn:
    result = conn.execute("SELECT * FROM actual_data LIMIT 5;")
    print("First 5 rows of actual_data:")
    for row in result:
        print(row)

# Verify forecast data
with engine.connect() as conn:
    result = conn.execute("SELECT * FROM forecast_data LIMIT 5;")
    print("\nFirst 5 rows of forecast_data:")
    for row in result:
        print(row)

            datum  Antiinflammatory  Antirheumatic  Analgesics  Antipyretics  \
2014-01-05   14.0             11.67           21.3      185.95          41.0   
2014-01-12  29.33             12.68           37.9      190.70          88.0   
2014-01-19  30.67             26.34           45.9      218.40          80.0   
2014-01-26   34.0             32.37           31.5      179.60          80.0   
2014-02-02  31.02             23.35           20.7      159.88          84.0   

            Psycholeptics  Sedatives  Bronchodilators  Antihistamines  
2014-01-05            0.0       32.0              7.0             NaN  
2014-01-12            5.0       21.0              7.2             NaN  
2014-01-19            8.0       29.0             12.0             NaN  
2014-01-26            8.0       23.0             10.0           34.00  
2014-02-02           12.0       29.0             12.0           31.02  
       datum  Antiinflammatory  Antirheumatic  Analgesics  Antipyretics  \
0 2019-10-20